# Redis Guest Book

## Reference

https://kubernetes.io/docs/tutorials/stateless-application/guestbook/#objectives

In [1]:
%env k=microk8s.kubectl

env: k=microk8s.kubectl


In [2]:
! $k version

Client Version: version.Info{Major:"1", Minor:"17", GitVersion:"v1.17.2", GitCommit:"59603c6e503c87169aea6106f57b9f242f64df89", GitTreeState:"clean", BuildDate:"2020-01-18T23:30:10Z", GoVersion:"go1.13.5", Compiler:"gc", Platform:"linux/amd64"}
Server Version: version.Info{Major:"1", Minor:"17", GitVersion:"v1.17.2", GitCommit:"59603c6e503c87169aea6106f57b9f242f64df89", GitTreeState:"clean", BuildDate:"2020-01-18T23:22:30Z", GoVersion:"go1.13.5", Compiler:"gc", Platform:"linux/amd64"}


# ! wget application/guestbook/redis-master-deployment.yaml 

```yaml
apiVersion: apps/v1 # for versions before 1.9.0 use apps/v1beta2
kind: Deployment
metadata:
  name: redis-master
  labels:
    app: redis
spec:
  selector:
    matchLabels:
      app: redis
      role: master
      tier: backend
  replicas: 1
  template:
    metadata:
      labels:
        app: redis
        role: master
        tier: backend
    spec:
      containers:
      - name: master
        image: k8s.gcr.io/redis:e2e  # or just image: redis
        resources:
          requests:
            cpu: 100m
            memory: 100Mi
        ports:
        - containerPort: 6379

```


In [3]:
! wget https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/application/guestbook/redis-master-deployment.yaml

--2020-03-08 05:08:38--  https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/application/guestbook/redis-master-deployment.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 586 [text/plain]
Saving to: ‘redis-master-deployment.yaml’

redis-master-deploy 100%[===================>]     586  --.-KB/s    in 0s      

2020-03-08 05:08:39 (97.2 MB/s) - ‘redis-master-deployment.yaml’ saved [586/586]



In [4]:
! cat redis-master-deployment.yaml

apiVersion: apps/v1 # for versions before 1.9.0 use apps/v1beta2
kind: Deployment
metadata:
  name: redis-master
  labels:
    app: redis
spec:
  selector:
    matchLabels:
      app: redis
      role: master
      tier: backend
  replicas: 1
  template:
    metadata:
      labels:
        app: redis
        role: master
        tier: backend
    spec:
      containers:
      - name: master
        image: k8s.gcr.io/redis:e2e  # or just image: redis
        resources:
          requests:
            cpu: 100m
            memory: 100Mi
        ports:
        - containerPort: 6379


In [7]:
! cat redis-master-deployment.yaml | egrep 'app:|Port|name|image'

  name: redis-master
    app: redis
      app: redis
        app: redis
      - name: master
        image: k8s.gcr.io/redis:e2e  # or just image: redis
        - containerPort: 6379


In [8]:
!   $k  apply -f https://k8s.io/examples/application/guestbook/redis-master-deployment.yaml

deployment.apps/redis-master created


In [9]:
! $k get deployments

NAME                  READY   UP-TO-DATE   AVAILABLE   AGE
kubernetes-bootcamp   1/1     1            1           28h
microbot              2/2     2            2           26h
product-deployment    2/2     2            2           34m
product-service       4/4     4            4           14h
redis-master          0/1     1            0           13s


In [10]:
! $k get deployments -o wide |grep redis

redis-master          1/1     1            1           41s   master                k8s.gcr.io/redis:e2e                           app=redis,role=master,tier=backend


In [11]:
! $k get pods -o wide

NAME                                   READY   STATUS    RESTARTS   AGE   IP          NODE   NOMINATED NODE   READINESS GATES
kubernetes-bootcamp-69fbc6f4cf-cmtnn   1/1     Running   0          28h   10.1.6.2    k8s    <none>           <none>
microbot-5bb7fd7f5d-8tf4t              1/1     Running   0          26h   10.1.6.8    k8s    <none>           <none>
microbot-5bb7fd7f5d-xrxd8              1/1     Running   0          26h   10.1.6.9    k8s    <none>           <none>
product-deployment-595f4456-mnwsj      1/1     Running   0          35m   10.1.6.15   k8s    <none>           <none>
product-deployment-595f4456-rkpjq      1/1     Running   0          35m   10.1.6.14   k8s    <none>           <none>
product-service-57f7889bd6-7jpxd       1/1     Running   0          14h   10.1.6.11   k8s    <none>           <none>
product-service-57f7889bd6-mvxtg       1/1     Running   0          14h   10.1.6.13   k8s    <none>           <none>
product-service-57f7889bd6-rfk7z       1/1     Running 

In [12]:
! $k delete deployment product-deployment

deployment.apps "product-deployment" deleted


In [14]:
! $k get pods -o wide

NAME                                   READY   STATUS    RESTARTS   AGE    IP          NODE   NOMINATED NODE   READINESS GATES
kubernetes-bootcamp-69fbc6f4cf-cmtnn   1/1     Running   0          28h    10.1.6.2    k8s    <none>           <none>
microbot-5bb7fd7f5d-8tf4t              1/1     Running   0          26h    10.1.6.8    k8s    <none>           <none>
microbot-5bb7fd7f5d-xrxd8              1/1     Running   0          26h    10.1.6.9    k8s    <none>           <none>
product-service-57f7889bd6-7jpxd       1/1     Running   0          14h    10.1.6.11   k8s    <none>           <none>
product-service-57f7889bd6-mvxtg       1/1     Running   0          14h    10.1.6.13   k8s    <none>           <none>
product-service-57f7889bd6-rfk7z       1/1     Running   0          14h    10.1.6.10   k8s    <none>           <none>
product-service-57f7889bd6-sv2gq       1/1     Running   0          14h    10.1.6.12   k8s    <none>           <none>
redis-master-7db7f6579f-b6cdk          1/1     

In [15]:
! $k delete po product-service

Error from server (NotFound): pods "product-service" not found


In [19]:
! $k get deployments

NAME                  READY   UP-TO-DATE   AVAILABLE   AGE
kubernetes-bootcamp   1/1     1            1           28h
microbot              2/2     2            2           26h
product-service       4/4     4            4           14h
redis-master          1/1     1            1           4m55s


In [20]:
! $k delete deployment product-service

deployment.apps "product-service" deleted


In [24]:
! $k get pods -o wide |grep redi

redis-master-7db7f6579f-b6cdk          1/1     Running   0          5m49s   10.1.6.16   k8s    <none>           <none>


In [31]:
%%bash
$k get pods -l 'app = redis'

NAME                            READY   STATUS    RESTARTS   AGE
redis-master-7db7f6579f-b6cdk   1/1     Running   0          11m


In [37]:
%%bash
podname=$($k get pods -l 'app = redis' | grep redis | cut -d " " -f1)
echo $podname
#$k logs -f $podname

redis-master-7db7f6579f-b6cdk


In [39]:
!$k logs redis-master-7db7f6579f-b6cdk

                _._                                                  
           _.-``__ ''-._                                             
      _.-``    `.  `_.  ''-._           Redis 2.8.19 (00000000/0) 64 bit
  .-`` .-```.  ```\/    _.,_ ''-._                                   
 (    '      ,       .-`  | `,    )     Running in stand alone mode
 |`-._`-...-` __...-.``-._|'` _.-'|     Port: 6379
 |    `-._   `._    /     _.-'    |     PID: 1
  `-._    `-._  `-./  _.-'    _.-'                                   
 |`-._`-._    `-.__.-'    _.-'_.-'|                                  
 |    `-._`-._        _.-'_.-'    |           http://redis.io        
  `-._    `-._`-.__.-'_.-'    _.-'                                   
 |`-._`-._    `-.__.-'    _.-'_.-'|                                  
 |    `-._`-._        _.-'_.-'    |                                  
  `-._    `-._`-.__.-'_.-'    _.-'                                   
      `-._    `-.__.-'    _.-'                               

In [40]:
! wget https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/application/guestbook/redis-master-service.yaml

--2020-03-08 05:28:00--  https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/application/guestbook/redis-master-service.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233 [text/plain]
Saving to: ‘redis-master-service.yaml’

redis-master-servic 100%[===================>]     233  --.-KB/s    in 0s      

2020-03-08 05:28:00 (12.5 MB/s) - ‘redis-master-service.yaml’ saved [233/233]



In [41]:
! $k apply -f https://k8s.io/examples/application/guestbook/redis-master-service.yaml

service/redis-master created


In [42]:
! $k get svc

NAME               TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)        AGE
kubernetes         ClusterIP   10.152.183.1     <none>        443/TCP        28h
microbot-service   NodePort    10.152.183.195   <none>        80:31914/TCP   26h
redis-master       ClusterIP   10.152.183.198   <none>        6379/TCP       9s


## Start up the Redis Slaves

Although the Redis master is a single pod, you can make it highly available to meet traffic demands by adding replica Redis slaves.
Creating

In [44]:
! wget https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/application/guestbook/redis-slave-deployment.yaml

--2020-03-08 05:30:32--  https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/application/guestbook/redis-slave-deployment.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1134 (1.1K) [text/plain]
Saving to: ‘redis-slave-deployment.yaml’

redis-slave-deploym 100%[===================>]   1.11K  --.-KB/s    in 0s      

2020-03-08 05:30:33 (215 MB/s) - ‘redis-slave-deployment.yaml’ saved [1134/1134]



In [45]:
!   $k apply -f https://k8s.io/examples/application/guestbook/redis-slave-deployment.yaml

deployment.apps/redis-slave created


In [47]:
! $k get pods |grep redis

redis-master-7db7f6579f-b6cdk          1/1     Running   0          19m
redis-slave-7664787fbc-fjnzc           1/1     Running   0          20s
redis-slave-7664787fbc-wh8qd           1/1     Running   0          20s


In [48]:
# redsis salve service
! wget https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/application/guestbook/redis-slave-service.yaml

--2020-03-08 05:32:15--  https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/application/guestbook/redis-slave-service.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209 [text/plain]
Saving to: ‘redis-slave-service.yaml’

redis-slave-service 100%[===================>]     209  --.-KB/s    in 0s      

2020-03-08 05:32:16 (38.6 MB/s) - ‘redis-slave-service.yaml’ saved [209/209]



In [49]:
! $k apply -f https://k8s.io/examples/application/guestbook/redis-slave-service.yaml

service/redis-slave created


In [50]:
! $k get svc

NAME               TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)        AGE
kubernetes         ClusterIP   10.152.183.1     <none>        443/TCP        28h
microbot-service   NodePort    10.152.183.195   <none>        80:31914/TCP   26h
redis-master       ClusterIP   10.152.183.198   <none>        6379/TCP       4m16s
redis-slave        ClusterIP   10.152.183.78    <none>        6379/TCP       13s


In [52]:
! $k get all --all-namespaces

NAMESPACE     NAME                                                  READY   STATUS    RESTARTS   AGE
default       pod/kubernetes-bootcamp-69fbc6f4cf-cmtnn              1/1     Running   0          28h
default       pod/microbot-5bb7fd7f5d-8tf4t                         1/1     Running   0          26h
default       pod/microbot-5bb7fd7f5d-xrxd8                         1/1     Running   0          26h
default       pod/redis-master-7db7f6579f-b6cdk                     1/1     Running   0          24m
default       pod/redis-slave-7664787fbc-fjnzc                      1/1     Running   0          5m1s
default       pod/redis-slave-7664787fbc-wh8qd                      1/1     Running   0          5m1s
kube-system   pod/coredns-9b8997588-xq4rf                           1/1     Running   0          28h
kube-system   pod/dashboard-metrics-scraper-687667bb6c-zn7pd        1/1     Running   0          28h
kube-system   pod/heapster-v1.5.2-5c58f64f8b-hdk4r                  4/4     Running   1  

In [53]:
! microk8s.status --wait-ready

microk8s is running
addons:
cilium: disabled
dashboard: enabled
dns: enabled
fluentd: disabled
gpu: disabled
helm: disabled
ingress: disabled
istio: disabled
jaeger: disabled
juju: disabled
knative: disabled
kubeflow: disabled
linkerd: disabled
metallb: disabled
metrics-server: disabled
prometheus: disabled
rbac: disabled
registry: disabled
storage: disabled


In [54]:
! microk8s.status --wait-ready | grep dns

dns: enabled


# DNS is important in multi-tier apps

In [55]:
# guestbook
! wget https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/application/guestbook/frontend-deployment.yaml

--2020-03-08 05:40:18--  https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/application/guestbook/frontend-deployment.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1107 (1.1K) [text/plain]
Saving to: ‘frontend-deployment.yaml’

frontend-deployment 100%[===================>]   1.08K  --.-KB/s    in 0s      

2020-03-08 05:40:18 (185 MB/s) - ‘frontend-deployment.yaml’ saved [1107/1107]



In [56]:
! $k apply -f https://k8s.io/examples/application/guestbook/frontend-deployment.yaml

deployment.apps/frontend created


In [57]:
! $k  get pods -l app=guestbook -l tier=frontend

NAME                        READY   STATUS              RESTARTS   AGE
frontend-6cb7f8bd65-hclq9   0/1     ContainerCreating   0          19s
frontend-6cb7f8bd65-lks7s   0/1     ContainerCreating   0          19s
frontend-6cb7f8bd65-sprzn   0/1     ContainerCreating   0          19s


# In these labs we use NodePort 

### in production we would use some loadbalancer front end

## now expose the front end service as a nodeport

In [59]:
! wget https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/application/guestbook/frontend-service.yaml

--2020-03-08 05:45:25--  https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/application/guestbook/frontend-service.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 438 [text/plain]
Saving to: ‘frontend-service.yaml’

frontend-service.ya 100%[===================>]     438  --.-KB/s    in 0s      

2020-03-08 05:45:25 (80.5 MB/s) - ‘frontend-service.yaml’ saved [438/438]



In [61]:
! $k apply -f https://k8s.io/examples/application/guestbook/frontend-service.yaml
! $k get svc

service/frontend created
NAME               TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)        AGE
frontend           NodePort    10.152.183.182   <none>        80:30537/TCP   1s
kubernetes         ClusterIP   10.152.183.1     <none>        443/TCP        29h
microbot-service   NodePort    10.152.183.195   <none>        80:31914/TCP   26h
redis-master       ClusterIP   10.152.183.198   <none>        6379/TCP       18m
redis-slave        ClusterIP   10.152.183.78    <none>        6379/TCP       14m


In [63]:
! $k get svc |grep frontend 

frontend           NodePort    10.152.183.182   <none>        80:30537/TCP   77s


In [64]:
! $k describe svc frontend

Name:                     frontend
Namespace:                default
Labels:                   app=guestbook
                          tier=frontend
Annotations:              kubectl.kubernetes.io/last-applied-configuration:
                            {"apiVersion":"v1","kind":"Service","metadata":{"annotations":{},"labels":{"app":"guestbook","tier":"frontend"},"name":"frontend","namespa...
Selector:                 app=guestbook,tier=frontend
Type:                     NodePort
IP:                       10.152.183.182
Port:                     <unset>  80/TCP
TargetPort:               80/TCP
NodePort:                 <unset>  30537/TCP
Endpoints:                10.1.6.19:80,10.1.6.20:80,10.1.6.21:80
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


In [65]:
! $k describe svc frontend |grep NodePort

Type:                     NodePort
NodePort:                 <unset>  30537/TCP


In [66]:
! curl localhost:30537

<html ng-app="redis">
  <head>
    <title>Guestbook</title>
    <link rel="stylesheet" href="//netdna.bootstrapcdn.com/bootstrap/3.1.1/css/bootstrap.min.css">
    <script src="https://ajax.googleapis.com/ajax/libs/angularjs/1.2.12/angular.min.js"></script>
    <script src="controllers.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/angular-ui-bootstrap/0.13.0/ui-bootstrap-tpls.js"></script>
  </head>
  <body ng-controller="RedisCtrl">
    <div style="width: 50%; margin-left: 20px">
      <h2>Guestbook</h2>
    <form>
    <fieldset>
    <input ng-model="msg" placeholder="Messages" class="form-control" type="text" name="input"><br>
    <button type="button" class="btn btn-primary" ng-click="controller.onRedis()">Submit</button>
    </fieldset>
    </form>
    <div>
      <div ng-repeat="msg in messages track by $index">
        {{msg}}
      </div>
    </div>
    </div>
  </body>
</html>
